In [3]:
import pandas as pd
import re
#import nltk

#from nltk.corpus import sentiwordnet as swn

In [8]:
상폐기업뉴스 = pd.read_csv('상폐기업번역파일.csv')
상폐기업뉴스.drop('뉴스기사본문전처리', axis=1, inplace=True)

In [85]:
a = [re.contains('[가-힣]', s) for s in 상폐기업뉴스['기사본문번역']]
a

In [6]:
상폐기업뉴스[상폐기업뉴스['기사본문번역'].str.contains('[가-힣]')]['기사본문번역']

264     상승과 하락을 반복하면서 변동성이 확대되는 주식시장 흐름에서는 손실을 줄이고 차익실...
265     Looking at the appearance of the stock market ...
266     상승과 하락을 반복하면서 변동성이 확대되는 주식시장 흐름에서는 손실을 줄이고 차익실...
766     Korean companies for global economic recovery ...
772     일반인들이 주식으로 손실을 보는 이유는 간단하다. Just a habit of ri...
774     Wrap account with consultative status with rae...
890     NetWave made a public announcement that it has...
1087    In the crisis of KOSDAQ chronic deficit manage...
1227    The possibility of delisting have reported tha...
1297    At one time East as an antenna famous Dongyang...
1386    Eoulim Networks announced that it has installe...
1960    Reporter Seo Hye-rim, the bankruptcy departmen...
1966    Reporter Hyun Yoon-kyung, Ssangyong Engineerin...
1967    The image of Ssangyong Engineering & Construct...
1968    Reporter Hyun Yoon-kyung, Ssangyong Engineerin...
1969    Ssangyong Engineering & Construction, which is...
1983    Liquidity crisis Ssangyong Engineering & Const...
1993    As Ssa

In [ ]:
상폐기업뉴스['기사본문번역토큰'] = 상폐기업뉴스['기사본문번역'].apply(lambda x: nltk.word_tokenize(x))
상폐기업뉴스['기사본문번역토큰'] = 상폐기업뉴스['기사본문번역토큰'].apply(lambda x: nltk.pos_tag(x))

In [12]:
def word_sentiment_calculator(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))

In [11]:
def sentence_sentiment_calculator(sent):
    tokens =  nltk.word_tokenize(sent)
    pos_tags = nltk.pos_tag(tokens)
    
    pos_score = 0
    neg_score = 0
    for word, tag in pos_tags:
        pos_score += word_sentiment_calculator(word, tag)[0]
        neg_score += word_sentiment_calculator(word, tag)[1]
    return (pos_score, neg_score)

In [13]:
## 긍정일경우 0 부정일경우 1
predicted = []

for i in range(len(상폐기업뉴스)):
    scores = sentence_sentiment_calculator(상폐기업뉴스.기사본문번역[i])
    
    if scores[0] >= scores[1]:
        predicted.append(0)
    else:
        predicted.append(1)

상폐기업뉴스['긍정부정분류'] = predicted

In [9]:
기업 = 상폐기업뉴스.groupby('기업').count()['긍정부정분류'].index
뉴스 = 상폐기업뉴스.groupby('기업').count()['긍정부정분류'].values
전체기사수 = pd.DataFrame(columns=['기업명', '기사수'])
전체기사수['기업명'] = 기업
전체기사수['기사수'] = 뉴스

부정기사기업 = 상폐기업뉴스[상폐기업뉴스.긍정부정분류==1].groupby('기업').count()['긍정부정분류'].index
부정기사뉴스 = 상폐기업뉴스[상폐기업뉴스.긍정부정분류==1].groupby('기업').count()['긍정부정분류'].values
부정기사수 = pd.DataFrame(columns=['기업명', '부정기사수'])
부정기사수['기업명'] = 부정기사기업
부정기사수['부정기사수'] = 부정기사뉴스

상폐기업 = pd.merge(전체기사수, 부정기사수, on=['기업명'], how='left')
상폐기업['부정기사비율'] = 상폐기업['부정기사수'] / 상폐기업['기사수']
상폐기업

KeyError: '긍정부정분류'

In [81]:
상폐기업[상폐기업.기사수 >10].describe()

,기사수,부정기사수,부정기사비율
count,105.000000,105.000000,105.000000
mean,24.676190,8.047619,0.327686
std,8.185935,4.234431,0.126682
min,11.000000,1.000000,0.076923
25%,19.000000,5.000000,0.250000
50%,25.000000,7.000000,0.320000
75%,29.000000,10.000000,0.388889
max,57.000000,23.000000,0.703704


In [79]:
상폐기업[상폐기업['부정기사비율'].isnull()]

,기업명,기사수,부정기사수,부정기사비율
26,비엔씨컴퍼니,1,NaN,NaN
31,승화프리텍,1,NaN,NaN
82,위너지스,3,NaN,NaN
94,제이에이치코오스,2,NaN,NaN


In [50]:
부정기사수 = 상폐기업뉴스[상폐기업뉴스.긍정부정분류==1].groupby('기업').count()['긍정부정분류'].values

In [32]:
data1 = pd.read_csv('../Step1-1_재무비율모델/재무비율.csv')
data1 = data1[data1.부실기업여부==1]
data1

,회사명,회계년도,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,폐지일자,부실기업여부
0,네이쳐글로벌,2009,74.492049,-49.803589,-3.371182,673.432622,6.028268,2011-01-08,1
1,네프로아이티,2009,-0.465154,19.338150,13.884726,157.344363,76.780711,2011-11-05,1
2,뉴젠아이씨티,2009,27.593677,-1075.333708,-28.342808,136.342262,40.747316,2011-04-29,1
3,트루아워,2009,52.368738,-19.102365,-25.573719,412.297157,33.436299,2011-10-25,1
4,디패션,2009,20.920391,-333.294194,-23.158301,170.052742,71.074703,2011-05-07,1
...,...,...,...,...,...,...,...,...,...
130,위너지스,2016,4.206434,-376.016367,-21.360033,105.778735,78.213923,2018-10-11,1
131,위노바,2016,28.671608,-89.404580,-9.113157,487.272885,63.716291,2018-03-09,1
132,트레이스,2016,3.857923,-35.621815,0.788938,238.537763,22.490823,2018-10-11,1
133,레이젠,2016,-3.874133,-5.069385,-9.353084,190.706311,117.325339,2018-10-11,1
